In [1]:
#requirements
!pip install sentencepiece
!pip install tokenizers
!pip install transformers
!pip install nltk

import urllib
import torch.nn as nn
import json
import torch
import pandas as pd
import numpy as np
import transformers
from transformers import BertTokenizer, BertForPreTraining, BertConfig, AdamW  

from transformers import AutoTokenizer, BertForMaskedLM
from transformers import TFBertForTokenClassification
from tokenizers import BertWordPieceTokenizer

from nltk.tokenize import WhitespaceTokenizer
import tensorflow as tf

import os
import re #regular expression

import string 
from pathlib import Path
from torch.utils.data import Dataset , DataLoader
# /https://huggingface.co/docs/transformers/model_doc/bert#transformers.TFBertForTokenClassification.**kwargs

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [4]:
#PARAMETERS
# special tokens
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"]
# special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = True
EPOCHS=25
ACCUM_STEPS=5
NUM_EPOCHS=100
BATCH_SIZE=4
SEED =42
RETURN_TENSORS = False
LEARNING_RATE = 5e-5

In [5]:
# Define tokenizer
TOKENIZER = BertTokenizer.from_pretrained('bert-base-cased')
# TOKENIZER = WhitespaceTokenizer()

OSError: Can't load tokenizer for 'bert-base-cased'. Make sure that:

- 'bert-base-cased' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'bert-base-cased' is not a path to a local directory with something else, in that case)

- or 'bert-base-cased' is the correct path to a directory containing relevant tokenizer files



In [ ]:
# Download data

##fn has unique paragraphs per row 409
url='https://raw.githubusercontent.com/MikeDoes/ETH_NLP_Project/main/fin_num_merged.json'
response = urllib.request.urlopen(url)
data_fn=json.loads(response.read())

##fn3 has duplicated paragraphs per number found 1100 original
url='https://raw.githubusercontent.com/MikeDoes/ETH_NLP_Project/main/FinNum-3_ConCall_dev.json'
response = urllib.request.urlopen(url)
data_fn3=json.loads(response.read())

In [25]:
# data_fn3

In [35]:
#PRE PROCESS
    
def convertdata_pd(data, tokenize_labels=False, tokenizer=TOKENIZER,return_tensors=RETURN_TENSORS):
    """
    Converts the json to a pd. 
    It can receive the data finNum (dics and shape 409)
    optionally it can be perform the tokenization on the labels already (only for data_fn and word tokenization)
    """
    data_rows=[]
    labels =[]
    #data_fn
    if data == data_fn:
        for i in range(len(data)):
            temp = []
            paragraph = data[i]["paragraph"]
            numbers = data[i]["entities"]
            for j in range(len(numbers)):
                num = numbers[j]["target_num"]
                category = numbers[j]["category"]
                offset_start = numbers[j]["offset_start"]
                if tokenize_labels==False:
                    temp.append({
                        "number":num,
                        "label":category,
                        "pos":offset_start})
                    labels.append({"labels":category})
                elif return_tensors == False:
                    category_token = tokenizer(list(category.split(" "))) #tokenize the category 
                    temp.append({
                        "number":num,
                        "label":category_token["input_ids"][0][1],
                        "pos":offset_start})
                    labels.append({"labels":category_token["input_ids"][0][1]})
                else:
                    category_token = tokenizer(list(category.split(" ")),return_tensors='tf') #tokenize the category
                    temp.append({
                        "number":num,
                        "label":category_token["input_ids"][0][1],
                        "pos":offset_start})
                    labels.append({"labels":category_token["input_ids"][0][1]})
                     # add the category. Warning: category can only be one word
            data_rows.append({
                "paragraph":paragraph,
                "labels":temp
            })
    #data_fn3
    elif data == data_fn3:
        for i in range(len(data)):
            paragraph=data[i]["paragraph"]
            target=data[i]["target_num"]
            category=data[i]["category"]
            offset_start=data[i]["offset_start"]
            offset_end=data[i]["offset_end"]
            data_rows.append({
                "paragraph":paragraph,
                "target":target,
                "category":category,
                "offset_start":offset_start,
                "offset_end":offset_end,
                "model_prediction_category":'',
                "model_prediction_entity":''
            })
    x = pd.DataFrame(data_rows).iloc[:,0]
    y = pd.DataFrame(data_rows).iloc[:,1]
    
    labels.append({"labels":0}) #decided for 0 as everything is already a token, therefore assumed NULL==0
    labels_uniq =[]
    if tokenize_labels == False:
        labels_uniq=pd.DataFrame(labels).labels.unique()
#todo check before 0 is a token for padding, so probably we can t add as 0 
    
    return pd.DataFrame(data_rows), x, y, labels_uniq

In [82]:
# run for data_fn
# data_rows, x ,y,labels=convertdata_pd(data_fn)
# data_rows, x ,y=convertdata_pd(data_fn,tokenize_labels=True)

##Example
data_rows, x ,y,cat =convertdata_pd(data_fn,tokenize_labels=False,return_tensors='tf')
# data_rows, x ,y,cat =convertdata_pd(data_fn,tokenize_labels=True,return_tensors='tf')

# cat has 10 categories

In [22]:
# Prep for token
def clean(par):
    """
    receives a paragraph and cleans its text
    """
    par
    par = par.lower() #remove capital letters
    par = par.replace("[^a-zA-Z]", " ") #remove non english characters
    # remove pontuation, as the punctuation i also converted into tokens
    for char in par:
        if char in string.punctuation:
            par = par.replace(char,"")
    return par
            
#todo implement for non english https://pypi.org/project/langdetect/
#todo keep %

In [ ]:
##Example
parpar = clean (data_rows.iloc[0,0])
parpar

In [56]:
# Construct target vector with tokenized target
def constructor(par,target,tokenizer=TOKENIZER, return_tensors=RETURN_TENSORS):
    """
    Args
    par: paragraph (not clean so we know the position) 
    targets: format [{'number': '80', 'label': 1168, 'pos': 373}]
    where label is already tokenized
    and returns a vector with 0s and the label in the word positioning
    """
    par_clean=clean(par)
    par_token=tokenizer(par_clean)["input_ids"]
    #initiates the vector with the same number of zeros from the clean paragraph
    labv = np.zeros(len(par_token)) 
    nubv = np.zeros(len(par_token))

    for i in range(len(target)):
        pos=target[i]["pos"] #get the position of the number
        spl = par[0:pos] #split till position of wanted number
        spl_clean = clean(spl) #now we clean the paragraph
        if return_tensors == False:
            spl_token = tokenizer(spl_clean)["input_ids"]
            labv[len(spl_token)-1] = target[i]["label"]  #replace the position of the number by the category. -1 to remove token=102 which is the end
            nubv[len(spl_token)-1] = target[i]["number"]
        else:
            spl_token = tokenizer(spl_clean, return_tensors='tf')["input_ids"]
            labv[len(spl_token[0])-1] = target[i]["label"]  #replace the position of the number by the category. -1 to remove token=102 which is the end
            nubv[len(spl_token[0])-1] = target[i]["number"]
    labv.tolist()
    if return_tensors == 'tf':
        par_token=tokenizer(par_clean,return_tensors='tf') #i cant just convert to tf #todo check if need to add
        #todo: check if we need to add max_length =512, truncation=True, padding ='max_length')above and how to handle
        labv=tf.convert_to_tensor(labv,np.float32)
        
#     print(spl_token[0])        
    return par_token, labv, nubv #todo

In [175]:
par,y_v,nub=constructor(x_3,y_3, return_tensors='tf')

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


ValueError: could not convert string to float: 'date'

In [136]:
# Construct target vector with tokenized target
def constructor2(par,target,categories,tokenizer=TOKENIZER, return_tensors=RETURN_TENSORS):
    """
    Args
    par: paragraph (not clean so we know the position from the number in the original paragraph) 
    targets: format [{'number': '80', 'label': 1168, 'pos': 373}]
    where label is already tokenized
    and returns a vector with 0s and the label in the word positioning
    """
    
    #clean the paragraph
    par_clean=clean(par)
    par_token=tokenizer(par_clean)["input_ids"]
    
    #initiates the vector with the same number of zeros from the clean paragraph
    y_vector = np.zeros(len(par_token)) 
    y_token_vector = np.zeros(len(par_token))

    #if the y (target) has still the original categories and not tokenized
    if type(target[0]['label'])==str:
        for i in range(len(target)):
            pos=target[i]["pos"] #get the position of the number
            spl = par[0:pos] #split till position of wanted number from the original paragraph
            spl_clean = clean(spl) #now we clean the paragraph until that position
            
            if return_tensors == False:
                spl_token = tokenizer(spl_clean)["input_ids"]
                size = len(spl_token)-1 #-1 to remove token=102 which is the end
            else:
                spl_token = tokenizer(spl_clean, return_tensors='tf')["input_ids"]
                size = len(spl_token[0])-1 #-1 to remove token=102 which is the end
            
            y_vector[size] = np.where(categories==target[i]["label"])[0][0]
            #replace the position of the category by the position of the category in the list. 
            y_token_vector[size] = target[i]["number"]
     
    else:
        for i in range(len(target)):
            pos=target[i]["pos"] #get the position of the number
            spl = par[0:pos] #split till position of wanted number
            spl_clean = clean(spl) #now we clean the paragraph
            if return_tensors == False:
                spl_token = tokenizer(spl_clean)["input_ids"]
                y_vector[spl_token] = target[i]["label"]  #replace the position of the number by the category. -1 to remove token=102 which is the end
                y_token_vector[len(spl_token)-1] = target[i]["number"]
            else:
                spl_token = tokenizer(spl_clean, return_tensors='tf')["input_ids"]
                y_vector[len(spl_token[0])-1] = target[i]["label"]  #replace the position of the number by the category. -1 to remove token=102 which is the end
                y_token_vector[len(spl_token[0])-1] = target[i]["number"]
    
    
    y_vector.tolist()
    if return_tensors == 'tf':
        par_token=tokenizer(par_clean,return_tensors='tf') #i cant just convert to tf #todo check if need to add
        #todo: check if we need to add max_length =512, truncation=True, padding ='max_length')above and how to handle
        y_vector=tf.convert_to_tensor(y_vector,np.float32)
        
#     print(spl_token[0])        
    return par_token, y_vector, y_token_vector #todo

In [137]:
#Example
x_3=x[3]
y_3=y[3]
par,y_v,y_token=constructor2(x_3,y_3,cat, return_tensors='tf')

In [26]:
data_rows.iloc[3,:] # for non tokenized data
# data_rows.iloc[0,1][0]["label"].numpy() # for tokenized data

x_3=x[3]
y_3=y[3]


In [212]:
##Model
model = TFBertForTokenClassification.from_pretrained('bert-base-cased',output_attentions=False, num_labels=10)

file bert-base-cased\config.json not found


OSError: Can't load config for 'bert-base-cased'. Make sure that:

- 'bert-base-cased' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'bert-base-cased' is not a path to a local directory with something else, in that case)

- or 'bert-base-cased' is the correct path to a directory containing a config.json file



In [ ]:
model.classifier = nn.Linear(model.config['hidden_size'], adapt_args.num_classes)

In [ ]:
model.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [185]:
#model.classifier = nn.Linear(85,85)



TypeError: 'BertConfig' object is not subscriptable

In [202]:
configg = model.config

In [210]:
config = BertConfig.from_pretrained(settings.BERT_NAME)

NameError: name 'settings' is not defined

In [ ]:
config = BertConfig.from_pretrained(settings.BERT_NAME)

In [204]:
type(configg)

transformers.models.bert.configuration_bert.BertConfig

In [205]:
configg["hidden_size"]

TypeError: 'BertConfig' object is not subscriptable

In [195]:
config.num_hidden_layers

NameError: name 'config' is not defined

In [143]:
inputs2 = par
#inputs2["labels"] = tf.reshape(tf.constant([1] * tf.size(y_v).numpy()), (-1, tf.size(y_v)))
inputs2["labels"] = tf.reshape(y_v, (-1, tf.size(y_v)))

In [206]:
model.fit(inputs2["input_ids"][0], inputs2["labels"][0],epochs=EPOCHS)

3/3 [==============================] - 54s 3s/step - loss: 1.3593


In [ ]:
model.save('bert-base-cased')

In [147]:
inputs2["input_ids"]

<tf.Tensor: shape=(1, 85), dtype=int32, numpy=
array([[  101,  3219,  1106, 11769,  8928,  1116,  2798,  2686,  1554,
         1214,  1857, 13081, 16509,  1620,  3775,  1111,  1103,  1148,
         1159,  7143,  3213,  1104,  1166,  1275,  3775,  1111,  1103,
         1214, 16112,  1106,  1429,  1121,   130,  1107,  1504,  1105,
        15869,  1412,  3389, 18717,  1517,  1254, 14544,  1506,  1103,
        11769,  8928, 12256,  1114,  1412,  2905,  3389, 18155,  2898,
         1167,  1190,  1405,  3775,  1137,  1695,  1106,  5787,  3775,
        11636,  1103, 24228,  1104, 11769,  8928,  1116,  3418,  5028,
         1105,  4518,  1366,  1107,   170,  2012,  2259,  2568, 18155,
         1700,  5273, 10351,   102]])>

In [173]:
inputs2["input_ids"]

<tf.Tensor: shape=(1, 85), dtype=int32, numpy=
array([[  101,  3219,  1106, 11769,  8928,  1116,  2798,  2686,  1554,
         1214,  1857, 13081, 16509,  1620,  3775,  1111,  1103,  1148,
         1159,  7143,  3213,  1104,  1166,  1275,  3775,  1111,  1103,
         1214, 16112,  1106,  1429,  1121,   130,  1107,  1504,  1105,
        15869,  1412,  3389, 18717,  1517,  1254, 14544,  1506,  1103,
        11769,  8928, 12256,  1114,  1412,  2905,  3389, 18155,  2898,
         1167,  1190,  1405,  3775,  1137,  1695,  1106,  5787,  3775,
        11636,  1103, 24228,  1104, 11769,  8928,  1116,  3418,  5028,
         1105,  4518,  1366,  1107,   170,  2012,  2259,  2568, 18155,
         1700,  5273, 10351,   102]])>

In [177]:
#predict
outputs=model(inputs2["input_ids"])

In [182]:
outputs[0][0]

<tf.Tensor: shape=(85, 10), dtype=float32, numpy=
array([[ 6.09996438e-01, -9.35568810e-02, -5.71563005e-01,
         1.82661340e-02, -1.14775114e-01, -9.57063586e-03,
        -6.90958798e-02,  2.07841471e-02, -2.98000634e-01,
         5.63239038e-01],
       [ 7.07624555e-02,  3.98517773e-02, -3.75171274e-01,
        -2.65033320e-02, -2.16003209e-01,  2.04018041e-01,
        -2.02192336e-01,  3.41116786e-02, -1.16547391e-01,
         2.69588470e-01],
       [ 1.98007345e-01,  3.65581781e-01, -1.33165359e-01,
         1.22175202e-01, -3.58866632e-01,  1.78980514e-01,
        -2.93947250e-01,  3.67936492e-01, -1.61682159e-01,
         1.02176547e-01],
       [-1.61044091e-01,  4.52278584e-01, -3.50163877e-01,
         3.44813198e-01, -3.31458867e-01, -1.06631733e-01,
         1.05732083e-02,  3.36906731e-01, -3.55275750e-01,
         3.53468478e-01],
       [-2.70794511e-01,  4.35088754e-01, -4.31613803e-01,
         1.94240287e-01,  3.01667899e-02,  3.51396084e-01,
        -4.06330109e

In [161]:
input_ids = tf.constant(TOKENIZER.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
outputs = model(input_ids)
scores = outputs[0]

In [162]:
input_ids

<tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[  101,  8667,   117,  1139,  3676,  1110, 10509,   102]])>

In [164]:
outputs

TFTokenClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 8, 10), dtype=float32, numpy=
array([[[-0.7028173 ,  0.0290671 ,  0.14094062,  0.11958211,
          0.4643381 ,  0.1557987 , -0.56651926, -0.43616956,
         -0.37233385, -0.16765007],
        [-0.7027081 ,  0.02906421,  0.1409397 ,  0.11956649,
          0.46438566,  0.15582941, -0.5665567 , -0.4362227 ,
         -0.37236378, -0.16769962],
        [-0.70276004,  0.02907233,  0.14094695,  0.11957465,
          0.46437028,  0.15582116, -0.5665365 , -0.43619305,
         -0.37234667, -0.16767907],
        [-0.7027638 ,  0.02906602,  0.14093913,  0.11956943,
          0.464367  ,  0.15582053, -0.5665381 , -0.43620262,
         -0.37236387, -0.16767898],
        [-0.70268804,  0.0290756 ,  0.1409707 ,  0.11956419,
          0.4643933 ,  0.15586093, -0.5665642 , -0.43623987,
         -0.3724041 , -0.16773063],
        [-0.7026604 ,  0.0290841 ,  0.14095993,  0.11956125,
          0.46440467,  0.15585516, -0.5665569 , -0.4362

In [ ]:
## Global logic till now
# data_fn >> json
# convertdata pd >> dataframes, tokenizer
# clean text
# constructor par, num >> transforms into y vector of tokenized labels
# prepare to model
#     inputs2 = par
#     inputs2["labels"] = tf.reshape(tf.constant([1] * tf.size(y_v).numpy()), (-1, tf.size(y_v)))

In [ ]:
###DATA FOR VASILIS
#original data

data_rows.iloc[3,:] # for non tokenized data
# data_rows.iloc[0,1][0]["label"].numpy() # for tokenized data

x_3=x[3]
y_3=y[3]

# convert and tokenize
par,y_v,nub=constructor(x_3,y_3, return_tensors='tf')


# print
print("PARAGRAPH",len(x_3), x_3)
print("TOKENIZED", len(par),par)
print("VECTOR NUMBERS",len(nub),nub)
print("VECTOR CATEGORIES",len(y_v),y_v)

PARAGRAPH 485 Turning to Optum's financial results. Full year 2018 revenues surpassed $100 billion for the first time. Revenue growth of over $10 billion for the year accelerated to 11% from 9% in 2017. And likewise our operating margins once again strengthened across the Optum portfolio with our overall operating earnings growing more than $1.5 billion or 23% to $8.2 billion reflecting the leverage of Optum's scale businesses and putting us in a strong baseline earnings position entering 2019.
TOKENIZED 3 {'input_ids': <tf.Tensor: shape=(1, 85), dtype=int32, numpy=
array([[  101,  3219,  1106, 11769,  8928,  1116,  2798,  2686,  1554,
         1214,  1857, 13081, 16509,  1620,  3775,  1111,  1103,  1148,
         1159,  7143,  3213,  1104,  1166,  1275,  3775,  1111,  1103,
         1214, 16112,  1106,  1429,  1121,   130,  1107,  1504,  1105,
        15869,  1412,  3389, 18717,  1517,  1254, 14544,  1506,  1103,
        11769,  8928, 12256,  1114,  1412,  2905,  3389, 18155,  2898,
 

In [ ]:
y_v = np.array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1,    0,
    0, 2,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3,
    0,    0,    0,    0,    0,    0, 4,    0, 3,    0, 1,    0,
    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
    0,    0,    0,    0,    0,    0,    0,    0, 3,    0,    0, 4,
    0, 2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1,
    0])

In [ ]:
y_v = tf.convert_to_tensor(y_v)

In [ ]:
inputs2 = par
#inputs2["labels"] = tf.reshape(tf.constant([1] * tf.size(y_v).numpy()), (-1, tf.size(y_v)))
inputs2["labels"] = tf.reshape(y_v, (-1, tf.size(y_v)))

In [ ]:
##MODEL information
#https://huggingface.co/docs/transformers/model_doc/bert#transformers.TFBertForTokenClassification.**kwargs
model = TFBertForTokenClassification.from_pretrained('bert-base-cased',output_attentions=False, num_labels=10)

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# options on how to train the model
# 1 https://stackoverflow.com/questions/62797376/tensorflow-bert-for-token-classification-exclude-pad-tokens-from-accuracy-whil
# model.fit(inputs2)
# 2 https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.TFPreTrainedModel.train_step
# model.train_step(inputs2)
# 3
# outputs = model(inputs2,output_hidden_states=True, return_dict =True, training = True)

In [ ]:
model.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(inputs2["input_ids"][0], inputs2["labels"][0])

3/3 [==============================] - 34s 2s/step - loss: 1.3685


In [ ]:
#optimizer = tf.optimizers.AdamW(learning_rate=lr, weight_decay=wd)



AttributeError: ignored

In [ ]:
# tips for model
# use AdamW optimizer
# you can get the loss via
loss = outputs.loss
logits = outputs.logits

#
for epoch in range(2):
    loop = tqdm(loader, leave = True)
    for batch in loop:
        optim.zero_grad()
        outputs = model(inputs)
        loss = output.loss
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Tips

In [ ]:
#how to get the different parts of tf
inputs.keys()

OLD - Other things

In [ ]:
###
# #save x to file
# with open('paragraphs.txt', 'w') as output:
#     output.write(x)

ftxt = x.to_csv('paragraphs.txt', index=False, sep='\n', mode='a')
#separate per line todo: check if we need a file per paragraph
    
# data_rows.to_csv('autonlp.csv', index=False)

In [ ]:
#MODEL

#option 1

class ModelBert:
    """
    Using the raw Bert Model 
    "https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209
    """
    def __init__(self):
        super(ModelBert,self).__init__()
        self.out = nn.Linear(self.bert.config.hidden_size,classes)
        self.sigmoid = nn.Sigmoid()
        #To get the size of the raw BERT outputs, simply use self.bert.config.hidden_size
        # we use the raw bert, without being for a specific task
        
    def forward(self,input,attention_mask):
        _,output = self.bert(input,attention_mask = attention_mask)
        #check attention mask
        res = self.sigmoid(self.out(output))
        return res
    
    def train(self):
        training_loss = []
        opmitizer.zero_grad()
        
    #def train(self):   
    #return True

#option 2
#https://huggingface.co/dslim/bert-base-NER
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
# obs: there are different pipeline options available from transformers.
nlp = pipeline("ner"), model=model) , tokenizer=tokenizer)

#option 3
model = BertForMaskedLM.from_pretrained('bert-base-uncased',    return_dict = True)
output = model(**a)

In [ ]:
b = convertdata_pd(data_fn)
example = b.iloc[0,0]
ner_results = nlp(example)